In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv("Youtube Video Dataset.csv")

In [3]:
df.head()

,Title,Videourl,Category,Description
0,Madagascar Street Food!!! Super RARE Malagasy ...,/watch?v=EwBA1fOQ96c,Food,🎥GIANT ALIEN SNAIL IN JAPAN! » https://youtu.b...
1,42 Foods You Need To Eat Before You Die,/watch?v=0SPwwpruGIA,Food,This is the ultimate must-try food bucket list...
2,Gordon Ramsay’s Top 5 Indian Dishes,/watch?v=upfu5nQB2ks,Food,We found 5 of the best and most interesting In...
3,How To Use Chopsticks - In About A Minute 🍜,/watch?v=xFRzzSF_6gk,Food,You're most likely sitting in a restaurant wit...
4,Trying Indian Food 1st Time!,/watch?v=K79bXtaRwcM,Food,HELP SUPPORT SINSTV!! Shop Our Sponsors!\nLast...


In [4]:
df.isnull().mean()

Title          0.000000
Videourl       0.000000
Category       0.000000
Description    0.007403
dtype: float64

In [5]:
df.dropna(inplace = True)

In [6]:
df.isnull().sum()

Title          0
Videourl       0
Category       0
Description    0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
import emoji 
import string

In [9]:
f = open('english_stopwords.txt','r')

In [10]:
stopwords = f.read()

In [11]:
stopwords = stopwords.split(',')

In [12]:
def preprocess(text):
    text = text.lower()
    #removing special chars
    text = text.replace('-','')
    text = text.replace('»','')
    text = text.replace('@','')
    text = text.replace('+','')
    text = text.replace('%','')
    text = text.replace('#','')
    text = text.replace('*','')
    text = text.replace('/','')
    text = emoji.replace_emoji(text,'')
    text = text.split()
    y = []
    for word in text:
        if 'http' not in word and 'https' not in word and 'www.' not in word and 'gmail' not in word:
            if word not in stopwords and word not in y and word not in string.punctuation:
                y.append(word)
    return " ".join(y)

In [13]:
df = df.sample(8000).reset_index().drop(columns = ['index'])

In [14]:
df['Description'] = df['Description'].apply(preprocess)

In [15]:

df.head()

,Title,Videourl,Category,Description
0,What is the Internet of Things? And why should...,/watch?v=_AlcRoqS65E,Science&Technology,"talk local tedx event, produced independently ..."
1,350 रु में 10 कुर्ती ! Kurti Manufacturer and ...,/watch?v=0f68JmeCHB0,manufacturing,350 रु में 10 कुर्ती kurti manufacturer wholes...
2,Boat Food around the World- Mexico City,/watch?v=bPAOGqDoVSU,Food,boat food world: mexico city's residents' favo...
3,DELL - How your laptop is build,/watch?v=5QC_QShyus0,manufacturing,video showing laptop build. raw parts final pr...
4,"Travel Vlog: Seoul, South Korea | HAUSOFCOLOR",/watch?v=9Zd8UjpJmK4,travel blog,seoul sooooo fun! tons quirky food lots shoppi...


In [16]:
df['tags'] =  df['Category'] + " "+df['Description']

In [17]:
df.head()

,Title,Videourl,Category,Description,tags
0,What is the Internet of Things? And why should...,/watch?v=_AlcRoqS65E,Science&Technology,"talk local tedx event, produced independently ...","Science&Technology talk local tedx event, prod..."
1,350 रु में 10 कुर्ती ! Kurti Manufacturer and ...,/watch?v=0f68JmeCHB0,manufacturing,350 रु में 10 कुर्ती kurti manufacturer wholes...,manufacturing 350 रु में 10 कुर्ती kurti manuf...
2,Boat Food around the World- Mexico City,/watch?v=bPAOGqDoVSU,Food,boat food world: mexico city's residents' favo...,Food boat food world: mexico city's residents'...
3,DELL - How your laptop is build,/watch?v=5QC_QShyus0,manufacturing,video showing laptop build. raw parts final pr...,manufacturing video showing laptop build. raw ...
4,"Travel Vlog: Seoul, South Korea | HAUSOFCOLOR",/watch?v=9Zd8UjpJmK4,travel blog,seoul sooooo fun! tons quirky food lots shoppi...,travel blog seoul sooooo fun! tons quirky food...


In [18]:
df.drop(columns=['Description'],inplace = True)

In [19]:
# import pickle
# pickle.dump(df.to_dict(),open("df_dict.pkl","wb"))

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf = TfidfVectorizer(max_features=5000)

In [22]:
vectors = tfidf.fit_transform(df['tags']).toarray()

In [23]:
similarity   = cosine_similarity(vectors)

In [24]:
similarity[0]

array([1.        , 0.06677378, 0.00277274, ..., 0.00533004, 0.04050102,
       0.01819157])

In [25]:
sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x: x[1])[1:6]

[(4534, 0.6742491038935268),
 (1591, 0.662124583203592),
 (1470, 0.6121769556787461),
 (1075, 0.6086607545587752),
 (3249, 0.607403560051018)]

In [26]:
similarity.shape

(8000, 8000)

In [27]:
shape = (8000,5)
simi = np.zeros(shape)

In [28]:
simi = [0,3]
# simi.shape
simi.pop(0)

0

In [29]:
simi.pop(0)

3

In [30]:
for i in range(0,similarity.shape[0]):
    simi.append(sorted(list(enumerate(similarity[i])),reverse = True, key = lambda x: x[1])[1:6])

In [31]:
sorted(list(simi[1]),reverse = True, key = lambda x: x[1])[0:5]

[(3419, 0.7198592310463281),
 (326, 0.3797086573441207),
 (6925, 0.36416584134796426),
 (605, 0.3425323911336283),
 (4453, 0.3335319323720615)]

In [32]:
sorted(list(enumerate(similarity[1])),reverse = True, key = lambda x: x[1])[1:6]

[(3419, 0.7198592310463281),
 (326, 0.3797086573441207),
 (6925, 0.36416584134796426),
 (605, 0.3425323911336283),
 (4453, 0.3335319323720615)]

In [46]:
def recommend(title):
    video_index = df[df['Title'] == title].index[0]
    dist = simi[video_index]
    video_list = sorted(list((dist)),reverse = True, key = lambda x: x[1])[0:5]
    for i in video_list:
        print(df.iloc[i[0]].Title)
        print("-"*50)

In [51]:
recommend(df['Title'][26])

Why It’s So Hard to Capture CO2 From the Air
--------------------------------------------------
This Quantum ‘Sixth Sense’ Could Allow Birds to Navigate Earth’s Magnetic Field
--------------------------------------------------
What Do Artificial Sweeteners Actually Do to Your Body?
--------------------------------------------------
Is Phosphorus the Reason We Haven’t Found Aliens Yet?
--------------------------------------------------
The ‘Ham Sandwich Theorem’ Will Change How You See the Universe… Seriously
--------------------------------------------------


['Why It’s So Hard to Capture CO2 From the Air',
 'This Quantum ‘Sixth Sense’ Could Allow Birds to Navigate Earth’s Magnetic Field',
 'What Do Artificial Sweeteners Actually Do to Your Body?',
 'Is Phosphorus the Reason We Haven’t Found Aliens Yet?',
 'The ‘Ham Sandwich Theorem’ Will Change How You See the Universe… Seriously']

In [50]:
df[df['Title']=='Why a Half Degree Rise in Global Temperature Would Be Catastrophic']

,Title,Videourl,Category,tags
26,Why a Half Degree Rise in Global Temperature W...,/watch?v=teZk3NLU_7M,Science&Technology,"Science&Technology report, earth reach 1.5°c w..."


In [35]:
df.sample(10)

,Title,Videourl,Category,tags
479,Gurudwara Manikaran Sahib and Hot Springs in K...,/watch?v=XZhXZdmrzpY,travel blog,travel blog gurudwara manikaran sahib pilgrima...
3597,My Solo Birthday Trip to Puerto Rico | All Thi...,/watch?v=tI6Ju0TiI_Q,travel blog,travel blog ♡ subscribe channel follow insta a...
593,"LAB RULES - Dua Lipa ""New Rules"" Parody",/watch?v=BRDApYgvDqQ,Science&Technology,Science&Technology low cost science parody dua...
4273,Top 10 Facts About The United Kingdom,/watch?v=Jrka9Nt-nQo,History,History 10 facts united kingdom! music: nicola...
6521,Rajasthan Diaries | Jaipur & Udaipur | Beautif...,/watch?v=Foch4KRL3fo,travel blog,travel blog jaipur diaries panna meena stepwel...
4228,"EKTA - An Arts, Music and Art of living Festiv...",/watch?v=XLhCx5YnNb8,Art&Music,"Art&Music ekta arts, music art living festival..."
869,Debunking All Those Personality Tests You Clic...,/watch?v=JN09RNHE0mI,Science&Technology,Science&Technology internet chockfull personal...
4467,"The emergence of ""4D printing"" | Skylar Tibbits",/watch?v=0gMCZFHv9v8,Science&Technology,Science&Technology printing grown sophisticati...
7417,Stretchy Batteries Are Coming... Here's How Th...,/watch?v=ZkzxtprYI_0,Science&Technology,Science&Technology researchers developed batte...
2715,கோவா பகுதி 2/ Part - 2 / Goa Vlog | #Travel #B...,/watch?v=Wo1Du-kHwfE,travel blog,travel blog கோவாவிற்கு செல்லவேண்டும் ஆனால் எப்...


In [42]:
import pickle

In [37]:
# pickle.dump(simi,open('simi.pkl'))
# pickle.dump(df.to_dict,open('df.pkl'))

In [49]:
pickle.dump(df.to_dict(),open('df.pkl','wb'))

In [53]:
pickle.dump(simi,open('simi1.pkl','wb'))